In [25]:
import pandas as pd
import numpy as np
import nltk
import time
import matplotlib.pyplot as plt
import seaborn as sns
import os, glob
import csv
from functools import reduce
import re
import string

In [26]:
import pandas as pd
import numpy as np
labels = ["Direct Harassment", "Hate Speech","Sexual Harassment","Trolling", "Others", "Toxic"]

batch_files = ['usanews.csv', 'foxnews0.csv', 'foxnews1.csv', 'foxnews2.csv', 'foxnews3.csv', 
                'nogla0.csv','pew0.csv', 'rae0_0.csv', 
                 'rae0_1.csv', 'terror0_0.csv', 'terror0_1.csv']

vid_files = ["foxnews_FULL.csv", "rae_FULL.csv",  "terror_FULL.csv",
             "usanews.csv" , "pew0.csv", "nogla0.csv"]
genre_files = ["gaming_channels_GENRE.csv", "news_channel_GENRE.csv" ]

files = []
files.extend(batch_files)
files.extend(vid_files)
files.extend(genre_files)
files = np.unique(np.array(files))

print(files)


['foxnews0.csv' 'foxnews1.csv' 'foxnews2.csv' 'foxnews3.csv'
 'foxnews_FULL.csv' 'gaming_channels_GENRE.csv' 'news_channel_GENRE.csv'
 'nogla0.csv' 'pew0.csv' 'rae0_0.csv' 'rae0_1.csv' 'rae_FULL.csv'
 'terror0_0.csv' 'terror0_1.csv' 'terror_FULL.csv' 'usanews.csv']


In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
def get_tfidf_vect(documents, a, b):
    vectorizer = TfidfVectorizer(ngram_range=(a,b), max_df=0.85, analyzer = 'word', min_df=1, max_features=10000, stop_words='english', lowercase=True)
    vector = vectorizer.fit_transform(documents)
    feature_names = np.array(vectorizer.get_feature_names())
    return pd.DataFrame(vector.todense(), columns=feature_names), vector, feature_names

In [28]:
def get_combined_tfidf_values_per_label(tfidf_df):
    combined_tfidf = pd.DataFrame(columns= (list(features) + labels))
    for label in labels:
        combined_scores = tfidf_df[tfidf_df[label] == 1][features].sum()
        to_append = list(combined_scores) + [0,0,0,0,0,0]
        row = pd.Series(to_append, index = (list(features) + labels))
        combined_tfidf = combined_tfidf.append(row, ignore_index=True)
        combined_tfidf.loc[combined_tfidf.index[-1],label] = 1.0
    return combined_tfidf

In [29]:
from scipy.stats import spearmanr
def get_corr_vals(df):
    features = df.columns.difference(labels)
    pvals = {}
    spm = {}
    for label in labels:
        p_values = {}
        sp_coeff = {}
        for f in features:
            #print(f)
            sp, pval = spearmanr(df[label], df[f])
            p_values[f] = pval
            sp_coeff[f] = sp
        pvals[label + " p-val"] = p_values
        spm[label + " corr coeff"] = sp_coeff
        
        x = pd.DataFrame.from_dict(pvals)
        y = pd.DataFrame.from_dict(spm)
        z = pd.concat([x,y], axis=1)
        z = z.reindex(sorted(z.columns), axis=1)
    return z


In [32]:
files = ["foxnews_FULL.csv", "rae_FULL.csv",  "terror_FULL.csv",
             "usanews.csv" , "pew0.csv", "nogla0.csv", "gaming_channels_GENRE.csv", "news_channel_GENRE.csv" ]

src_folder = "../Annotations/CSVs/"
dest_folder = "tfidf_scores_WORDS_per_video/"

for file in files:
    #read file
    df = pd.read_csv(src_folder + file, index_col = 0)
    #get words, POS SEQ
    df["words"] = df["words"].replace(np.nan, '', regex=True)
    
    #tfidf of words
    tfidf_df, vector, features = get_tfidf_vect(df['words'], 2,3)
    tfidf_df[labels] = df[labels] #add tthe labels
    
    #combine tfidf values per label
    combined_tfidf = get_combined_tfidf_values_per_label(tfidf_df)
    combined_tfidf.to_csv(dest_folder + file)
    

# Word/s Correlation

In [37]:
src_folder = "tfidf_scores_WORDS_per_video/"
dest_folder = "corr_words_per_label/"
for file in files:
    #read file
    df = pd.read_csv(src_folder + file, index_col = 0)
    df = get_corr_vals(df)
    df.to_csv("corr_words_per_label/" + file)

C:\Users\Phoebe\anaconda3\lib\site-packages\scipy\stats\stats.py:4196: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(SpearmanRConstantInputWarning())


In [35]:

dest_folder = "corr_words_per_label/"
test = pd.read_csv(dest_folder + files[0], index_col = 0)
display(test.head(50))

,Direct Harassment corr coeff,Direct Harassment p-val,Hate Speech corr coeff,Hate Speech p-val,Others corr coeff,Others p-val,Sexual Harassment corr coeff,Sexual Harassment p-val,Toxic corr coeff,Toxic p-val,Trolling corr coeff,Trolling p-val
000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000 ntd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000 plus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000 plus watch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10 pct,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10 pct usa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100 free,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100 free fair,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
src_folder = "tfidf_scores_WORDS_per_video/"
dest_folder = "corr_words_per_label/"
test = pd.read_csv(src_folder + files[0], index_col = 0)
# x = get_corr_vals(test)
# display(x.head(50))
test

# POS Tag Correlation

In [21]:
files = ["foxnews_FULL.csv", "rae_FULL.csv",  "terror_FULL.csv",
             "usanews.csv" , "pew0.csv", "nogla0.csv", "gaming_channels_GENRE.csv", "news_channel_GENRE.csv" ]

src_folder = "../Annotations/CSVs/"
dest_folder = "tfidf_scores_POS_per_video/"

for file in files:
    #read file
    df = pd.read_csv(src_folder + file, index_col = 0)
    #get words, POS SEQ
    df["POS-SEQ"] = df["POS-SEQ"].replace(np.nan, '', regex=True)
    
    #tfidf of words
    tfidf_df, vector, features = get_tfidf_vect(df['POS-SEQ'], 1,4)
    tfidf_df[labels] = df[labels] #add tthe labels
    
    #combine tfidf values per label
    combined_tfidf = get_combined_tfidf_values_per_label(tfidf_df)
    combined_tfidf.to_csv(dest_folder + file)
    

In [24]:
src_folder = "tfidf_scores_POS_per_video/"
dest_folder = "corr_pos_seq_per_label/"

for file in files:
    #read file
    df = pd.read_csv(src_folder + file, index_col = 0)
    df = get_corr_vals(df)
    df.to_csv(dest_folder + file)